In [25]:
import pandas as pd

import json
import glob
import os
import re

In [26]:
base = "../exp/"
paths = glob.glob(base + "applications/*.json")
data = [json.load(open(path)) for path in paths]

def path_to_graph(path):
    return {
        'ger06': 'TDGer06',
        'ptv17': 'TDEur17',
        'ptv20': 'TDEur20',
        'osm_europe': 'OSM Europe',
        'osm_ger': 'OSM Ger',
        'osm_ger_td': 'OSM Ger',
        'europe': 'DIMACs Eur',
    }[[x for x in path.split('/') if x != ''][-1]]

In [27]:
algos = ['Virtual Topocore Component Query', 'Virtual Topocore Bidirectional Core Query']
queries = pd.DataFrame.from_records([{ 
    **algo,
    'exp': exp['experiment'], 
    'graph': path_to_graph(run['args'][1]),
    'num_queue_pops': algo.get('num_queue_pops', 0) + algo.get('core_search', {}).get('num_queue_pops', 0),
    'num_queue_pushs': algo.get('num_queue_pushs', 0) + algo.get('core_search', {}).get('num_queue_pushs', 0),
    'num_relaxed_arcs': algo.get('num_relaxed_arcs', 0) + algo.get('core_search', {}).get('num_relaxed_arcs', 0)
    } for run in data for exp in run.get('experiments', []) for algo in exp['algo_runs'] if (algo.get('algo') in algos or ('algo' not in algo and 'biggest_bcc_size' not in algo))])

queries = queries.append(pd.DataFrame.from_records([{ 
    **algo,
    'exp': run['program'], 
    'graph': path_to_graph(run['args'][1]),
    'num_queue_pops': algo.get('num_queue_pops', 0) + algo.get('core_search', {}).get('num_queue_pops', 0),
    'num_queue_pushs': algo.get('num_queue_pushs', 0) + algo.get('core_search', {}).get('num_queue_pushs', 0),
    'num_relaxed_arcs': algo.get('num_relaxed_arcs', 0) + algo.get('core_search', {}).get('num_relaxed_arcs', 0)
    } for run in data for algo in run.get('algo_runs', []) if algo.get('algo') in algos or ('algo' not in algo and 'biggest_bcc_size' not in algo)]))

In [28]:
algos = ['Dijkstra Query', 'Bidrectional Dijkstra Query']
dijkstra_queries = pd.DataFrame.from_records([{ 
    'exp': exp['experiment'], 
    'graph': path_to_graph(run['args'][1]),
    **algo } 
    for run in data for exp in run.get('experiments', []) for algo in exp['algo_runs'] if algo.get('algo') in algos])

dijkstra_queries = dijkstra_queries.append(pd.DataFrame.from_records([{ 
    'exp': run['program'], 
    'graph': path_to_graph(run['args'][1]),
    **algo } 
    for run in data for algo in run.get('algo_runs', []) if algo.get('algo') in algos]))

In [29]:
queries['affected'] = queries['lower_bound'] != queries['result']
queries['increase'] = (queries['result'] / queries['lower_bound'] - 1) * 100

In [30]:
queries.groupby(['graph', 'exp']).count()

algo  core_search   from  lower_bound  \
graph      exp                                                           
DIMACs Eur perfect              10000        10000  10000        10000   
OSM Ger    bidir_cchpot_turns   10000            0  10000        10000   
           bidir_chpot_live     10000            0  10000        10000   
           bidir_chpot_turns    10000            0  10000        10000   
           bidir_no_highways    10000            0  10000        10000   
           bidir_no_tunnels     10000            0  10000        10000   
           cchpot_live          10000        10000  10000        10000   
           cchpot_turns          9040         8950  10000         9040   
           chpot_live           10000        10000  10000        10000   
           chpot_td             10000        10000  10000        10000   
           chpot_td_live        10000        10000  10000        10000   
           chpot_turns           9040         8950  10000         9040   
           chpot_turns_td_live   9069         8979  10000         9069   
           no_highways          10000         9968  10000        10000   
           no_tunnels           10000         9993  10000        10000   
           perfect              10000        10000  10000        10000   
TDEur17    chpot_td             10000        10000  10000        10000   
TDEur20    chpot_td             10000         9999  10000        10000   
TDGer06    chpot_td             10000        10000  10000        10000   

                                num_queue_pops  num_queue_pushs  \
graph      exp                                                    
DIMACs Eur perfect                       10000            10000   
OSM Ger    bidir_cchpot_turns            10000            10000   
           bidir_chpot_live              10000            10000   
           bidir_chpot_turns             10000            10000   
           bidir_no_highways             10000            10000   
           bidir_no_tunnels              10000            10000   
           cchpot_live                   10000            10000   
           cchpot_turns                  10000            10000   
           chpot_live                    10000            10000   
           chpot_td                      10000            10000   
           chpot_td_live                 10000            10000   
           chpot_turns                   10000            10000   
           chpot_turns_td_live           10000            10000   
           no_highways                   10000            10000   
           no_tunnels                    10000            10000   
           perfect                       10000            10000   
TDEur17    chpot_td                      10000            10000   
TDEur20    chpot_td                      10000            10000   
TDGer06    chpot_td                      10000            10000   

                                num_relaxed_arcs  result  running_time_ms  \
graph      exp                                                              
DIMACs Eur perfect                         10000   10000            10000   
OSM Ger    bidir_cchpot_turns              10000    8168            10000   
           bidir_chpot_live                10000   10000            10000   
           bidir_chpot_turns               10000    8168            10000   
           bidir_no_highways               10000    9522            10000   
           bidir_no_tunnels                10000    9954            10000   
           cchpot_live                     10000   10000            10000   
           cchpot_turns                    10000    8168            10000   
           chpot_live                      10000   10000            10000   
           chpot_td                        10000   10000            10000   
           chpot_td_live                   10000   10000            10000   
           chpot_turns                     10000    8168            1

In [41]:
table = queries.groupby(['graph', 'exp']).mean()[['running_time_ms', 'num_queue_pushs', 'increase']]
table['num_queue_pushs'] = table['num_queue_pushs'] / 1000
table = table.join(dijkstra_queries.groupby(['graph', 'exp']).agg(dijkstra_running_time_ms=('running_time_ms', 'mean')))
table['speedup'] = table['dijkstra_running_time_ms'] / table['running_time_ms']
table = table.round(1)
table = table.reset_index()
table['algo'] = table['exp'].map({
    'perfect': 'CH U',
    'chpot_td': 'CH U',
    'chpot_live': 'CH U',
    'chpot_turns': 'CH U',
    'chpot_td_live': 'CH U',
    'chpot_turns_td_live': 'CH U',
    'no_highways': 'CH U',
    'no_tunnels': 'CH U',
    'cchpot_live': 'CCH U',
    'cchpot_turns': 'CCH U',
    'bidir_cchpot_turns': 'CCH B',
    'bidir_chpot_live': 'CH B',
    'bidir_chpot_turns': 'CH B',
    'bidir_no_highways': 'CH B',
    'bidir_no_tunnels': 'CH B',
})
table['app'] = table['exp'].map({
    'perfect': 'Unmodified $w_q = w_{\ell}$',
    'chpot_td': 'TD',
    'chpot_live': 'Live',
    'chpot_turns': 'Turns',
    'chpot_td_live': 'TD + Live',
    'chpot_turns_td_live': 'TD + Live + Turns',
    'no_highways': 'No Highways',
    'no_tunnels': 'No Tunnels',
    'cchpot_live': 'Live',
    'cchpot_turns': 'Live + Turns',
    'bidir_cchpot_turns': 'Live + Turns',
    'bidir_chpot_live': 'Live',
    'bidir_chpot_turns': 'Turns',
    'bidir_no_highways': 'No Highways',
    'bidir_no_tunnels': 'No Tunnels',
})
table = table.set_index(['graph', 'app', 'algo']) \
    .reindex(index=['Unmodified $w_q = w_{\ell}$', 'No Tunnels', 'No Highways', 'Live', 'Turns', 'Live + Turns', 'TD', 'TD + Live', 'TD + Live + Turns'], level=1) \
    .reindex(index=['CH U', 'CH B', 'CCH U', 'CCH B'], level=2) \
    .drop(columns=['exp'])
print(table.to_latex(escape=False))
table

\begin{tabular}{lllrrrrr}
\toprule
        &    &      &  running_time_ms &  num_queue_pushs &  increase &  dijkstra_running_time_ms &  speedup \\
graph & app & algo &                  &                  &           &                           &          \\
\midrule
DIMACs Eur & Unmodified $w_q = w_{\ell}$ & CH U &              0.9 &              1.1 &       0.0 &                    2106.0 &   2405.8 \\
OSM Ger & Unmodified $w_q = w_{\ell}$ & CH U &              0.6 &              0.5 &       0.0 &                    2182.6 &   3795.4 \\
        & No Tunnels & CH U &             29.2 &             46.8 &       5.2 &                    2198.0 &     75.2 \\
        &    & CH B &             33.4 &             35.7 &       5.2 &                    1532.0 &     45.9 \\
        & No Highways & CH U &            378.7 &            583.8 &      42.5 &                    1992.5 &      5.3 \\
        &    & CH B &            433.1 &            481.6 &      42.5 &                    1721.4 &    

running_time_ms  \
graph      app                         algo                     
DIMACs Eur Unmodified $w_q = w_{\ell}$ CH U               0.9   
OSM Ger    Unmodified $w_q = w_{\ell}$ CH U               0.6   
           No Tunnels                  CH U              29.2   
                                       CH B              33.4   
           No Highways                 CH U             378.7   
                                       CH B             433.1   
           Live                        CH U             129.4   
                                       CH B             193.6   
                                       CCH U              1.1   
           Turns                       CH U               3.0   
                                       CH B               1.1   
           Live + Turns                CCH U              4.8   
                                       CCH B              2.1   
           TD                          CH U             120.8   
           TD + Live                   CH U             198.3   
           TD + Live + Turns           CH U             474.2   
TDEur17    TD                          CH U              80.4   
TDEur20    TD                          CH U              97.7   
TDGer06    TD                          CH U               4.2   

                                              num_queue_pushs  increase  \
graph      app                         algo                               
DIMACs Eur Unmodified $w_q = w_{\ell}$ CH U               1.1       0.0   
OSM Ger    Unmodified $w_q = w_{\ell}$ CH U               0.5       0.0   
           No Tunnels                  CH U              46.8       5.2   
                                       CH B              35.7       5.2   
           No Highways                 CH U             583.8      42.5   
                                       CH B             481.6      42.5   
           Live                        CH U             193.9      15.0   
                                       CH B             188.8      15.0   
                                       CCH U              0.8       0.0   
           Turns                       CH U               5.7       1.1   
                                       CH B               0.8       1.1   
           Live + Turns                CCH U              8.8       1.0   
                                       CCH B              1.6       1.1   
           TD                          CH U             104.4      12.3   
           TD + Live                   CH U             170.3      20.7   
           TD + Live + Turns           CH U             657.8      21.7   
TDEur17    TD                          CH U              79.8       3.9   
TDEur20    TD                          CH U              72.8       4.2   
TDGer06    TD                          CH U               6.4       3.1   

                                              dijkstra_running_time_ms  \
graph      app                         algo                              
DIMACs Eur Unmodified $w_q = w_{\ell}$ CH U                     2106.0   
OSM Ger    Unmodified $w_q = w_{\ell}$ CH U                     2182.6   
           No Tunnels                  CH U                     2198.0   
                                       CH B                     1532.0   
           No Highways                 CH U                     1992.5   
                                       CH B                     1721.4   
           Live                        CH U                     2119.3   
                                       CH B                     1812.1   
                                       CCH U                    2341.3   
           Turns                       CH U                     4708.2   
                                       CH B                     2882.9   
           Live + Turns                CCH U                    4621.8   
                                       CCH B                    3274

In [14]:
queries.query('exp == "cchpot_live"')['increase'].describe()

count    10000.000000
mean         0.251448
std          0.462309
min          0.000000
25%          0.000000
50%          0.099557
75%          0.328084
max         11.284656
Name: increase, dtype: float64

In [7]:
table = queries.groupby(['graph', 'exp']).agg(
    running_time_ms=('running_time_ms', 'mean'), 
    num_pot_evals=('num_pot_evals', 'mean'), 
    num_relaxed_arcs=('num_relaxed_arcs', 'mean'),
    increase=('increase', 'mean'),
    affected=('affected', 'sum'),
    size=('affected', 'count'))
table['affected'] = table['affected'] * 100 / table['size']
table = table[table.columns[0:-1]]
table['num_relaxed_arcs'] = table['num_relaxed_arcs'] / 1000
table['num_pot_evals'] = table['num_pot_evals'] / 1000
table = table.join(preprocessing.groupby('graph').mean())
table = table.join(dijkstra_queries.groupby(['graph', 'exp']).agg(dijkstra_running_time_ms=('running_time_ms', 'mean')))
#table = table.round(1)
print(table.to_latex())
table

\begin{tabular}{llrrrrrrr}
\toprule
                                         &            &  running\_time\_ms &  num\_pot\_evals &  num\_relaxed\_arcs &  increase &  affected &  preprocessing\_running\_time\_s &  dijkstra\_running\_time\_ms \\
graph & exp &                  &                &                   &           &           &                               &                           \\
\midrule
/algoDaten/graphs/cleaned\_td\_road\_data/de/day/dido/ & chpot\_td &         4.341473 &       7.621476 &         30.077662 &  1.030244 &     98.23 &                     59.086588 &                604.205920 \\
/algoDaten/graphs/cleaned\_td\_road\_data/ptv17-eur-car/day/di/ & chpot\_td &        85.371324 &     103.096165 &        500.442628 &  1.038714 &     99.48 &                    292.520587 &               3245.881700 \\
/algoDaten/zeitz/roadgraphs/europe/ & fastest\_times\_10 &        50.767913 &     111.816654 &        390.435148 &  1.028436 &     96.28 &                    273.

running_time_ms  \
graph                                              exp                                 
/algoDaten/graphs/cleaned_td_road_data/de/day/d... chpot_td                 4.341473   
/algoDaten/graphs/cleaned_td_road_data/ptv17-eu... chpot_td                85.371324   
/algoDaten/zeitz/roadgraphs/europe/                fastest_times_10        50.767913   
                                                   fastest_times_2          6.696532   
                                                   random_times_10          0.975721   
                                                   random_times_2           0.970561   
/algoDaten/zeitz/roadgraphs/osm_ger_rel/           chpot_live             107.523830   
                                                   chpot_turns             38.234487   
                                                   no_highways            315.260351   
                                                   no_tunnels              29.906702   

                                                                     num_pot_evals  \
graph                                              exp                               
/algoDaten/graphs/cleaned_td_road_data/de/day/d... chpot_td               7.621476   
/algoDaten/graphs/cleaned_td_road_data/ptv17-eu... chpot_td             103.096165   
/algoDaten/zeitz/roadgraphs/europe/                fastest_times_10     111.816654   
                                                   fastest_times_2       13.683046   
                                                   random_times_10        1.127394   
                                                   random_times_2         1.111212   
/algoDaten/zeitz/roadgraphs/osm_ger_rel/           chpot_live           218.899497   
                                                   chpot_turns           85.649110   
                                                   no_highways          661.362835   
                                                   no_tunnels            60.373990   

                                                                     num_relaxed_arcs  \
graph                                              exp                                  
/algoDaten/graphs/cleaned_td_road_data/de/day/d... chpot_td                 30.077662   
/algoDaten/graphs/cleaned_td_road_data/ptv17-eu... chpot_td                500.442628   
/algoDaten/zeitz/roadgraphs/europe/                fastest_times_10        390.435148   
                                                   fastest_times_2          42.021788   
                                                   random_times_10           2.908221   
                                                   random_times_2            2.864245   
/algoDaten/zeitz/roadgraphs/osm_ger_rel/           chpot_live              801.182286   
                                                   chpot_turns             178.109398   
                                                   no_highways            2465.660474   
                                                   no_tunnels              224.124574   

                                                                     increase  \
graph                                              exp                          
/algoDaten/graphs/cleaned_td_road_data/de/day/d... chpot_td          1.030244   
/algoDaten/graphs/cleaned_td_road_data/ptv17-eu... chpot_td          1.038714   
/algoDaten/zeitz/roadgraphs/europe/                fastest_times_10  1.028436   
                                                   fastest_times_2   1.006781   
                                                   random_times_10   1.000056   
                                                   random_times_2    1.000007   
/algoDaten/zeitz/roadgraphs/osm_ger_rel/           chpot_live        1.137614   
                                                   chpot_turns       1.058078   
                                                   no_highways       1.392460   
                   